# tem Beneficios a entrega contínua?

## Revisar literatura para encontrar melhores perguntas de pesquisa

## Analisar dataset para estabelecer quais projetos tem entrega contínua



In [72]:
import pandas as pd
from pymongo import MongoClient
from bson.objectid import ObjectId
client = MongoClient()
db = client.smartshark_2_1

### 1) Tem integração contínua - OK

In [73]:
vcs_systems = pd.DataFrame(list(db.vcs_system.find()))
travisBuilds = pd.DataFrame(list(db.travis_build.find()))
projetosComBuilds = travisBuilds['vcs_system_id'].unique()
print(vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna().count())
vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()

_id                40
url                40
project_id         40
repository_type    40
last_updated       40
repository_file    40
dtype: int64


,_id,url,project_id,repository_type,last_updated,repository_file
0,58bfd79dfa94f2c7f0d32551,https://github.com/apache/commons-math,58bfd5860da3ef7cef464c78,git,2018-02-15 11:35:53.876,5dcbf5ce2ecf182bf5d37773
3,5b110f43328aa9d53131e747,https://github.com/apache/zeppelin.git,5b110ee90da3ef05a99f0de6,git,2018-06-01 11:17:55.880,5dcbf6142ecf182bf5d38124
4,5b1fa04730a71b06bc3734c1,https://github.com/apache/mahout,5b1f9fbb0da3ef49b7d0a480,git,2018-06-12 12:28:23.953,5dcbf61c2ecf182bf5d382c4
5,5b23df3c30a71b06bc01e16a,https://github.com/apache/struts.git,5b2225520da3ef3e52ece5e2,git,2018-06-15 17:46:04.170,5dcbf6222ecf182bf5d383fa
6,5b27c64830a71b06bceb0ead,https://github.com/apache/opennlp.git,5b27c4cc0da3ef789daec0a1,git,2018-06-18 16:48:40.415,5dcbf6262ecf182bf5d3852f
9,5b34a9c830a71b06bc3d9ef3,https://github.com/apache/calcite.git,5b34a9210da3ef789cc240ec,git,2018-06-28 11:26:32.261,5dcbf6362ecf182bf5d38708
10,5b3b305530a71b06bc5bba5b,https://github.com/apache/cayenne.git,5b3b2d520da3ef789ec953ea,git,2018-07-03 10:14:13.878,5dcbf63c2ecf182bf5d387d8
12,5b43737430a71b06bcbdd168,https://github.com/apache/kylin.git,5b43728b0da3ef07106a790f,git,2018-07-09 16:38:44.747,5dcbf6552ecf182bf5d3891f
14,5b47693d30a71b06bc04ba2e,https://github.com/apache/parquet-mr.git,5b4769050da3ef0712a85126,git,2018-07-12 16:44:13.340,5dcbf6652ecf182bf5d38fb6
20,5b76bead30a71b06bcadc156,https://github.com/apache/tez.git,5b76be770da3ef049126bdba,git,2018-08-17 14:25:17.050,5dcbf6982ecf182bf5d397ff


### 2) Pegar as tags do projetos de integração contínua? - OK

In [74]:
tags = pd.DataFrame(list(db.tag.find()))
idsVcsComBuilds = vcs_systems.query('_id.isin(@projetosComBuilds)').drop_duplicates().dropna()._id

In [75]:
tagsDosProjComBuild = tags.query('vcs_system_id.isin(@idsVcsComBuilds)').drop_duplicates().dropna()

In [76]:
tagsDosProjComBuild

,_id,name,commit_id,vcs_system_id,message,tagger_id,date,date_offset
0,58bfd7a53cff546cd0b42c99,MATH_3_4_RC1,58bfd7a53cff546cd0b42c95,58bfd79dfa94f2c7f0d32551,Creating Apache Commons Math v3.4 RC1 tag.\n--...,58bfd79d3cff546cc7b42a28,2014-12-18 20:30:14,60.0
1,58bfd7a53cff546cd3b42b9e,MATH_3_4_1_RC1,58bfd7a53cff546cd3b42b9a,58bfd79dfa94f2c7f0d32551,Creating Apache Commons Math v3.4.1 RC1 tag.\n...,58bfd79d3cff546cc7b42a28,2015-01-08 16:51:54,60.0
2,58bfd7a53cff546cd5b42bb7,MATH_3_4_RC3,58bfd7a53cff546cd5b42bb3,58bfd79dfa94f2c7f0d32551,Creating Apache Commons Math v3.4 RC3 tag.\n--...,58bfd79d3cff546cc7b42a28,2014-12-23 09:29:32,60.0
3,58bfd7a53cff546cd5b42bb9,MATH_3_4,58bfd7a53cff546cd5b42bb3,58bfd79dfa94f2c7f0d32551,RC3 becomes the 3.4 official version.\n-----BE...,58bfd79d3cff546cc7b42a28,2014-12-26 18:17:44,60.0
4,58bfd7a53cff546cd8b42c1b,MATH_3_4_RC2,58bfd7a53cff546cd8b42c17,58bfd79dfa94f2c7f0d32551,Creating Apache Commons Math v3.4 RC2 tag.\n--...,58bfd79d3cff546cc7b42a28,2014-12-19 14:41:18,60.0
...,...,...,...,...,...,...,...,...
3912,608c41384cecfade6c2d010e,mifosplatform-0.12.0.beta,608c41384cecfade6c2d010c,608c3a6cc26a57681e5764e8,mifosplatform 0.12.0\n,608c404e4cecfade6c2ba78f,2013-04-22 10:38:51,60.0
3913,608c428e4cecfade6c2e9adc,1.3.1-unreleased,608c428e4cecfade6c2e9ada,608c3a6cc26a57681e5764e8,see FINERACT-877\n,58c11ba602ca40f8bfb1f533,2020-10-04 15:36:08,120.0
3929,60a24b000f99f9deee0f0ea8,v2.3.27-gae,60a24b000f99f9deee0f0ea6,60a24ad9c26a57681e786a7b,Tagging v2.3.27-gae\n,58c1133302ca40f8bfb1e2ba,2017-11-02 15:25:18,-240.0
3930,60a24b000f99f9deee0f0eb6,v2.3.27,60a24b000f99f9deee0f0eb4,60a24ad9c26a57681e786a7b,Tagging v2.3.27\n,58c1133302ca40f8bfb1e2ba,2017-11-02 15:21:49,-240.0


### 3) Determinei oq vou considerar Tag de Release - OK
##### Tags de release não tem no nome os termos 'RC', 'candidate', 'test', 'beta', 'alfa', 'unreleased', snapshot, siteupdate, freeze, sandbox, migration

In [77]:
tagsDeRelase = tagsDosProjComBuild[
    tagsDosProjComBuild['name']
    .str.contains('rc|RC|candidate|CANDIDATE|test|TEST|beta|BETA|alpha|ALPHA|unreleased|UNRELEASED|snapshot|SNAPSHOT|siteupdate|SITEUPDATE|freeze|FREEZE|sandbox|SANDBOX|migration|MIGRATION')==False].drop_duplicates().dropna()

In [78]:
tagsDeRelase.name.to_csv('tags.csv', index=False)

In [79]:
tagsDeRelase

,_id,name,commit_id,vcs_system_id,message,tagger_id,date,date_offset
3,58bfd7a53cff546cd5b42bb9,MATH_3_4,58bfd7a53cff546cd5b42bb3,58bfd79dfa94f2c7f0d32551,RC3 becomes the 3.4 official version.\n-----BE...,58bfd79d3cff546cc7b42a28,2014-12-26 18:17:44,60.0
5,58bfd7b83cff546cc8b43dab,MATH_1_2,58bfd7b83cff546cc8b43da7,58bfd79dfa94f2c7f0d32551,MATH_1_2\n,58bfd7a53cff546ccbb42b96,2008-02-24 05:56:08,0.0
9,58bfd7b93cff546ccfb43cde,MATH_3_3,58bfd7b93cff546ccfb43cda,58bfd79dfa94f2c7f0d32551,MATH_3_3\n,58bfd79d3cff546cc7b42a28,2014-05-14 18:42:03,0.0
11,58bfd7c33cff546cd1b447ff,MATH_3_1,58bfd7c33cff546cd1b447fb,58bfd79dfa94f2c7f0d32551,MATH_3_1\n,58bfd7a53cff546ccab42b89,2012-12-23 18:09:22,0.0
13,58bfd7cd3cff546cd5b44d22,MATH_3_5,58bfd7cd3cff546cd5b44d1c,58bfd79dfa94f2c7f0d32551,RC2 becomes the 3.5 official version.\n-----BE...,58bfd79d3cff546cc7b42a28,2015-04-17 14:39:07,120.0
...,...,...,...,...,...,...,...,...
3910,608c3c6c4cecfade6c2684ff,1.0.0,608c3c6c4cecfade6c2684fd,608c3a6cc26a57681e5764e8,Fineract 1.0.0\n,608c3a764cecfade6c23bbcd,2017-05-19 01:09:19,330.0
3911,608c40b74cecfade6c2c4fc0,mifosplatform-1.9.0-tag,608c40b74cecfade6c2c4fbe,608c3a6cc26a57681e5764e8,mifos platform 1.9.0\n,608c404e4cecfade6c2ba78f,2013-09-09 08:27:49,60.0
3929,60a24b000f99f9deee0f0ea8,v2.3.27-gae,60a24b000f99f9deee0f0ea6,60a24ad9c26a57681e786a7b,Tagging v2.3.27-gae\n,58c1133302ca40f8bfb1e2ba,2017-11-02 15:25:18,-240.0
3930,60a24b000f99f9deee0f0eb6,v2.3.27,60a24b000f99f9deee0f0eb4,60a24ad9c26a57681e786a7b,Tagging v2.3.27\n,58c1133302ca40f8bfb1e2ba,2017-11-02 15:21:49,-240.0


### 4) Filtrar builds que tem commits de tag de release - OK

In [80]:
commitsDeRelease = tagsDeRelase.commit_id.unique()
buildsIntegrados = travisBuilds.query('commit_id.isin(@commitsDeRelease)')
stateIndesejados = ['failed', 'canceled']
BuildsValidos = buildsIntegrados.query('~state.isin(@stateIndesejados)')

In [81]:
BuildsValidos.state.value_counts()

passed     207
errored     47
Name: state, dtype: int64

### 5) Analisar quais projetos tem builds de entrega (34 no total) ok 

In [82]:
vcsComEntrega = BuildsValidos.vcs_system_id.unique()
print(vcs_systems.query('_id.isin(@vcsComEntrega)').drop_duplicates().dropna().count())
vcs_systems.query('_id.isin(@vcsComEntrega)').drop_duplicates().dropna()

_id                30
url                30
project_id         30
repository_type    30
last_updated       30
repository_file    30
dtype: int64


,_id,url,project_id,repository_type,last_updated,repository_file
5,5b23df3c30a71b06bc01e16a,https://github.com/apache/struts.git,5b2225520da3ef3e52ece5e2,git,2018-06-15 17:46:04.170,5dcbf6222ecf182bf5d383fa
6,5b27c64830a71b06bceb0ead,https://github.com/apache/opennlp.git,5b27c4cc0da3ef789daec0a1,git,2018-06-18 16:48:40.415,5dcbf6262ecf182bf5d3852f
9,5b34a9c830a71b06bc3d9ef3,https://github.com/apache/calcite.git,5b34a9210da3ef789cc240ec,git,2018-06-28 11:26:32.261,5dcbf6362ecf182bf5d38708
10,5b3b305530a71b06bc5bba5b,https://github.com/apache/cayenne.git,5b3b2d520da3ef789ec953ea,git,2018-07-03 10:14:13.878,5dcbf63c2ecf182bf5d387d8
12,5b43737430a71b06bcbdd168,https://github.com/apache/kylin.git,5b43728b0da3ef07106a790f,git,2018-07-09 16:38:44.747,5dcbf6552ecf182bf5d3891f
14,5b47693d30a71b06bc04ba2e,https://github.com/apache/parquet-mr.git,5b4769050da3ef0712a85126,git,2018-07-12 16:44:13.340,5dcbf6652ecf182bf5d38fb6
20,5b76bead30a71b06bcadc156,https://github.com/apache/tez.git,5b76be770da3ef049126bdba,git,2018-08-17 14:25:17.050,5dcbf6982ecf182bf5d397ff
22,5b7ffe9530a71b06bc70c038,https://github.com/apache/storm.git,5b7ffd4e0da3ef049126bdbd,git,2018-08-24 14:48:21.397,5dcbf6c52ecf182bf5d39bf3
24,5bbe0f4057674ee16793f245,https://github.com/apache/commons-lang.git,5bbe0efb0da3ef0ddd6d593d,git,2018-10-10 16:40:00.593,5dcbf6d62ecf182bf5d3a067
25,5bc5d34957674ee16701b68a,https://github.com/apache/pdfbox.git,5bc5cf050da3ef0de20a7284,git,2018-10-16 14:02:17.970,5dcbf6e72ecf182bf5d3a194


### 6) Desses projetos quais tem uma quantidade relevante de entregas que permita a nossa análise (>3 Builds com integração) (21 projetos no total)

In [83]:
BuildsValidosPorCommit = BuildsValidos[['_id', 'vcs_system_id', 'commit_id', 'number', 'state', 'started_at', 'finished_at']].groupby(['commit_id'], as_index=False, sort=False).min()

In [84]:
vcsComCD = BuildsValidosPorCommit[['vcs_system_id','_id']].groupby(['vcs_system_id'], as_index=False).agg('count').sort_values(by=['_id'], ascending=False)
vcsComCD = vcsComCD.query('_id > 2')
print(vcsComCD.count())
vcsComCD

vcs_system_id    20
_id              20
dtype: int64


,vcs_system_id,_id
11,5bd0268235e3ea2b7bbfdbae,17
4,5b43737430a71b06bcbdd168,13
0,5b23df3c30a71b06bc01e16a,12
7,5b7ffe9530a71b06bc70c038,9
1,5b27c64830a71b06bceb0ead,7
14,5bed61cc35e3ea2b7b2a1805,6
24,5d8ccfb33dfe5ddb8590d841,6
20,5c3f0a7c621a9a77b35f3add,6
28,608b9b33c26a57681ebf90b6,5
23,5cd2806d504acf99a4cd728c,5


### 7) Percorrer os projetos e coletar seus dados

In [85]:
vcsList= []
for index, row in vcsComCD.iterrows():
    vcsList.append(row.vcs_system_id)
vcsList

[ObjectId('5bd0268235e3ea2b7bbfdbae'),
 ObjectId('5b43737430a71b06bcbdd168'),
 ObjectId('5b23df3c30a71b06bc01e16a'),
 ObjectId('5b7ffe9530a71b06bc70c038'),
 ObjectId('5b27c64830a71b06bceb0ead'),
 ObjectId('5bed61cc35e3ea2b7b2a1805'),
 ObjectId('5d8ccfb33dfe5ddb8590d841'),
 ObjectId('5c3f0a7c621a9a77b35f3add'),
 ObjectId('608b9b33c26a57681ebf90b6'),
 ObjectId('5cd2806d504acf99a4cd728c'),
 ObjectId('5bbe0f4057674ee16793f245'),
 ObjectId('5c87a96c504acf99a4de5a18'),
 ObjectId('5d91bc213dfe5ddb85cc1e0e'),
 ObjectId('5bed83ca35e3ea2b7be8d7c9'),
 ObjectId('5b3b305530a71b06bc5bba5b'),
 ObjectId('5bee7d5b35e3ea2b7ba03bfa'),
 ObjectId('5c87a755504acf99a4dbf833'),
 ObjectId('5bc5d34957674ee16701b68a'),
 ObjectId('5b76bead30a71b06bcadc156'),
 ObjectId('5b47693d30a71b06bc04ba2e')]

Montar a primeira tabela, que para cada vcs_system_id, busca project_id, issues_system_id, data da adoção de CD, primeira build de CD, quantas builds antes e depois do CD, campo de análise manual se tem a CD suspeita

In [86]:
issue_system = pd.DataFrame(list(db.issue_system.find()))
project_ids = []
issues_system_ids = []
for vcs in vcsList:
    # Buscar informações sobre o projeto
    project_id = vcs_systems.query('_id == @vcs').iloc[0,2]    
    project_ids.append(project_id)

    # Informações sobre o issue system
    issues_system_id = issue_system.query('project_id == @project_id').iloc[0,1]
    issues_system_ids.append(issues_system_id)

    # Pegar o commit da build considerada como a build de adoção
    #BuildsValidosPorCommit.query('vcs_system_id == @a').sort_values(by=['started_at']).iloc[1,5]

    # Pegar a release com o commit de build de adoção

    # Montar dataframe
project_ids, issues_system_ids



([ObjectId('5bd022e30da3ef1a10738359'),
  ObjectId('5b43728b0da3ef07106a790f'),
  ObjectId('5b2225520da3ef3e52ece5e2'),
  ObjectId('5b7ffd4e0da3ef049126bdbd'),
  ObjectId('5b27c4cc0da3ef789daec0a1'),
  ObjectId('5bed583a0da3ef21a58b337c'),
  ObjectId('5d8cb4a10da3ef0b68077801'),
  ObjectId('5c3f09db0da3ef512137483f'),
  ObjectId('608b9b0227ad48a9ceb48654'),
  ObjectId('5cd27fe20da3ef671cda02e7'),
  ObjectId('5bbe0efb0da3ef0ddd6d593d'),
  ObjectId('5c87a9510da3ef6c89b69038'),
  ObjectId('5d91bbb20da3ef35cfd2f443'),
  ObjectId('5bed83440da3ef21a2107882'),
  ObjectId('5b3b2d520da3ef789ec953ea'),
  ObjectId('5bee7a9d0da3ef21a3a094b1'),
  ObjectId('5c87a7300da3ef6c8a4bf6d8'),
  ObjectId('5bc5cf050da3ef0de20a7284'),
  ObjectId('5b76be770da3ef049126bdba'),
  ObjectId('5b4769050da3ef0712a85126')],
 [ObjectId('5bd022e30da3ef1a10738359'),
  ObjectId('5b43728b0da3ef07106a790f'),
  ObjectId('5b2225520da3ef3e52ece5e2'),
  ObjectId('5b7ffd4e0da3ef049126bdbd'),
  ObjectId('5b27c4cc0da3ef789daec0a1'),

### 8) Buscar as issues dos projetos analisados para contabilizar as metricas

### 9) Contabilizar métricas

### 10) Buscar os commits dos projetos analisados para contabilizar a quantidade de membros